In [1]:
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
from pprint import pprint

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [5]:
from nltk.corpus import stopwords

In [7]:
import spacy

In [8]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [13]:
df = pd.read_csv('reviews.csv')
print(df.rating.unique())
df

[50 40 10 30 20]


,rating,review
0,50,We stayed at San Rafael for 3 night in March 2...
1,50,Walking off the street into the first courtyar...
2,50,San Rafael is very conveniently located in the...
3,50,Beautiful colonial-style hotel. We loved stayi...
4,50,"The hotel is beautiful,it reflects an old Anti..."
...,...,...
17470,50,My husband & I were there a couple of weeks ag...
17471,30,The Hotel itself is beautiful. I was very impr...
17472,50,Absolutly beautiful atmosphere. Green surroun...
17473,40,The hotel Casa Santo Domingo was one of the mo...


In [15]:
# Convertir a una lista
data = df.review.values.tolist()
pprint(data[:1])

['We stayed at San Rafael for 3 night in March 2020.  Our room  was great and '
 'the service was amazing!!!  They suggested wonderful restaurants ans a great '
 'walking tour to start off our trip... The hotel grounds are private and '
 'beautiful perfectly located within Antigua!  The stay surpassed any '
 'expectations we had and highly recommend to anybody visiting Antigua!!!']


In [16]:
# Eliminar emails - Todo lo que este antes y despues del arroba
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
pprint(data[:1])

['We stayed at San Rafael for 3 night in March 2020.  Our room  was great and '
 'the service was amazing!!!  They suggested wonderful restaurants ans a great '
 'walking tour to start off our trip... The hotel grounds are private and '
 'beautiful perfectly located within Antigua!  The stay surpassed any '
 'expectations we had and highly recommend to anybody visiting Antigua!!!']


In [17]:
# Eliminar newlines
data = [re.sub(r'\s+', ' ', sent) for sent in data]
pprint(data[:1])

['We stayed at San Rafael for 3 night in March 2020. Our room was great and '
 'the service was amazing!!! They suggested wonderful restaurants ans a great '
 'walking tour to start off our trip... The hotel grounds are private and '
 'beautiful perfectly located within Antigua! The stay surpassed any '
 'expectations we had and highly recommend to anybody visiting Antigua!!!']


In [18]:
# Eliminar comillas
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['We stayed at San Rafael for 3 night in March 2020. Our room was great and '
 'the service was amazing!!! They suggested wonderful restaurants ans a great '
 'walking tour to start off our trip... The hotel grounds are private and '
 'beautiful perfectly located within Antigua! The stay surpassed any '
 'expectations we had and highly recommend to anybody visiting Antigua!!!']


In [19]:
def sent_to_words(sentences):
    for sentence in sentences:
    # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 
    # deacc=True elimina la puntuación
data_words = list(sent_to_words(data))

print(data_words[:1])

[['we', 'stayed', 'at', 'san', 'rafael', 'for', 'night', 'in', 'march', 'our', 'room', 'was', 'great', 'and', 'the', 'service', 'was', 'amazing', 'they', 'suggested', 'wonderful', 'restaurants', 'ans', 'great', 'walking', 'tour', 'to', 'start', 'off', 'our', 'trip', 'the', 'hotel', 'grounds', 'are', 'private', 'and', 'beautiful', 'perfectly', 'located', 'within', 'antigua', 'the', 'stay', 'surpassed', 'any', 'expectations', 'we', 'had', 'and', 'highly', 'recommend', 'to', 'anybody', 'visiting', 'antigua']]


In [20]:
# Construimos modelos de bigrams y trigrams
# https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [21]:
print(bigram_mod[data_words[0]])
print(trigram_mod[bigram_mod[data_words[0]]])

['we', 'stayed', 'at', 'san_rafael', 'for', 'night', 'in', 'march', 'our', 'room', 'was', 'great', 'and', 'the', 'service', 'was', 'amazing', 'they', 'suggested', 'wonderful', 'restaurants', 'ans', 'great', 'walking', 'tour', 'to', 'start', 'off', 'our', 'trip', 'the', 'hotel', 'grounds', 'are', 'private', 'and', 'beautiful', 'perfectly', 'located', 'within', 'antigua', 'the', 'stay', 'surpassed', 'any', 'expectations', 'we', 'had', 'and', 'highly', 'recommend', 'to', 'anybody', 'visiting', 'antigua']
['we', 'stayed', 'at', 'san_rafael', 'for', 'night', 'in', 'march', 'our', 'room', 'was', 'great', 'and', 'the', 'service', 'was', 'amazing', 'they', 'suggested', 'wonderful', 'restaurants', 'ans', 'great', 'walking', 'tour', 'to', 'start', 'off', 'our', 'trip', 'the', 'hotel', 'grounds', 'are', 'private', 'and', 'beautiful', 'perfectly', 'located', 'within', 'antigua', 'the', 'stay', 'surpassed', 'any', 'expectations', 'we', 'had', 'and', 'highly_recommend', 'to', 'anybody', 'visiting', 

In [22]:
# Eliminar stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Hacer bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Hacer trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Lematización basada en el modelo de POS de Spacy
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [23]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

In [24]:
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [25]:
# python3 -m spacy download en_core_web_lg
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente
nlp = spacy.load('C:\\Users\\ergarciap\\Miniconda3\\Lib\\site-packages\\en_core_web_lg\\en_core_web_lg-2.3.0', disable=['parser', 'ner'])

# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['stay', 'night', 'room', 'service', 'amazing', 'suggest', 'wonderful', 'restaurant', 'great', 'walking', 'tour', 'start', 'trip', 'hotel', 'ground', 'private', 'beautiful', 'perfectly', 'locate', 'surpass', 'expectation', 'highly', 'recommend', 'visit']]


In [26]:
# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)

In [27]:
for key, value in id2word.items():
    print(key, value)

0 amazing
1 beautiful
2 expectation
3 great
4 ground
5 highly
6 hotel
7 locate
8 night
9 perfectly
10 private
11 recommend
12 restaurant
13 room
14 service
15 start
16 stay
17 suggest
18 surpass
19 tour
20 trip
21 visit
22 walking
23 wonderful
24 able
25 accent
26 bad
27 beautifully_decorated
28 bottle
29 breakfast
30 choice
31 comfortable
32 different
33 flower
34 fountain
35 fresh
36 get
37 greatly_appreciate
38 helpful
39 long
40 lush
41 make
42 menu
43 need
44 nice
45 planting
46 recommendation
47 shopping
48 spanish
49 spectacular
50 staff
51 suggestion
52 try
53 variety
54 wait
55 walk
56 well
57 wine
58 wish
59 world
60 attentive
61 conveniently_locate
62 day
63 friendly
64 heart
65 immaculately
66 keep
67 last
68 pleasure
69 property
70 return
71 week
72 wife
73 back
74 can
75 coffee
76 courtyard
77 fall
78 feature
79 go
80 love
81 nearby
82 plant
83 receptionist
84 relax
85 sound
86 staying
87 surround
88 water
89 work
90 arc
91 block
92 location
93 old
94 personal
95 reflect


1149 unhappy
1150 dedication
1151 sheer
1152 splendid
1153 therefore
1154 active
1155 ash
1156 birthday
1157 cloud
1158 gray
1159 locale
1160 poof
1161 rebooke
1162 recall
1163 stratovolcano
1164 advance
1165 communication
1166 map
1167 orientation
1168 pm
1169 point
1170 previously
1171 process
1172 reusable
1173 several
1174 supermarket
1175 tuk_tuk
1176 via_email
1177 years_ago
1178 description
1179 museum
1180 seclude
1181 dull
1182 english
1183 equipped
1184 side
1185 sofa
1186 somewhat
1187 appoint
1188 exceptionally
1189 flowering_plant
1190 forward
1191 solid
1192 straight
1193 con
1194 dim
1195 english_speake
1196 flood
1197 intrude
1198 lightbulb
1199 locked_gate
1200 machine
1201 pro
1202 wattage
1203 crazy
1204 geranium
1205 plaza
1206 store
1207 sweet
1208 like
1209 parking
1210 relaxing
1211 bloom
1212 buzz
1213 case
1214 creepy
1215 fire
1216 former
1217 hummingbird
1218 opposite
1219 solemn
1220 stand
1221 vine
1222 worker
1223 guard
1224 please
1225 upgrade
1226 dinnin

2148 hop
2149 jump
2150 skip
2151 cinder
2152 echo
2153 horrible
2154 wise
2155 equidistant
2156 extraordinarily
2157 fold
2158 obliging
2159 cachet
2160 dead
2161 perfict
2162 cognizant
2163 mosquito
2164 soundproof
2165 birds_chirpe
2166 ceiling
2167 gripe
2168 post
2169 skim
2170 soak
2171 spread
2172 excited
2173 inviting
2174 loft
2175 perfectly_manicure
2176 remnant
2177 absorb
2178 chapel
2179 def
2180 excite
2181 nook
2182 phenomenal
2183 archway
2184 deck
2185 flor
2186 sights
2187 team
2188 appart
2189 congregate
2190 sheltered
2191 engagement
2192 mariachis
2193 video
2194 force
2195 fuzzy
2196 impressed
2197 welcoming
2198 bnb
2199 sophisticated
2200 tank
2201 tepid
2202 longtime
2203 photographer
2204 vistas
2205 custom
2206 strike
2207 vista
2208 woodworking
2209 mark
2210 nothing_fancy
2211 phone_call
2212 route
2213 bicycle
2214 cancel
2215 chicken
2216 chocolate
2217 dismay
2218 nicuraga
2219 raw
2220 soothe
2221 soup
2222 throat
2223 altough
2224 beautifull
2225 boss


3810 insanely
3811 accurate
3812 enable
3813 locking
3814 parque_centrale
3815 approximately
3816 aka
3817 cantaloupe
3818 hybrid
3819 caprese
3820 downgrade
3821 hammer
3822 hammering
3823 ongoing
3824 reluctant
3825 spite
3826 undertake
3827 language_barrier
3828 fade
3829 forested
3830 hillside
3831 lavender
3832 scent
3833 billow
3834 studying_spanish
3835 dormitory
3836 omelette
3837 convinient
3838 abode
3839 elses
3840 environ
3841 faint
3842 hole
3843 homage
3844 impede
3845 regrettably
3846 stargazer
3847 watering
3848 rose
3849 wiifi
3850 stretch
3851 critical
3852 bookshelf
3853 demeanor
3854 seemingly
3855 exited
3856 smiling
3857 backyard
3858 empanada
3859 privacy_curtain
3860 doorway
3861 grill
3862 kebab
3863 mingle
3864 processional
3865 ethnic
3866 buidling
3867 circuit
3868 higher
3869 warrant
3870 avoidant
3871 hostile
3872 indifferent
3873 involvement
3874 raise
3875 scenario
3876 suspicion
3877 theft
3878 latch
3879 deffinitly
3880 vego
3881 belt
3882 bountiful
38

5398 lukewarm
5399 primary
5400 idiotic
5401 pillar
5402 shush
5403 farytale
5404 wholesome
5405 ultra
5406 endlessly
5407 mood
5408 pub_crawl
5409 tom
5410 roofdeck
5411 strikingly
5412 scale
5413 touristed
5414 likewise
5415 understate
5416 previous_reviews
5417 stereo
5418 subject
5419 tear
5420 ad
5421 advertising
5422 expand
5423 integrate
5424 archaeological
5425 entree
5426 lamb
5427 connected
5428 centrally_located
5429 cleanse
5430 lost
5431 responsibility
5432 thab
5433 unsympathetic
5434 strap
5435 beef
5436 gouge
5437 incessantly
5438 inconvenience
5439 pee
5440 piss
5441 squawk
5442 enthusiasm
5443 layover
5444 redo
5445 cupoboard
5446 jumping
5447 assign
5448 pendant
5449 commune
5450 souvenir
5451 defenitely
5452 frienidly
5453 weighted
5454 cough
5455 dehumidifier
5456 rainy
5457 react
5458 relocate
5459 wheeze
5460 appearance
5461 contract
5462 curio
5463 ironing
5464 municipality
5465 distueb
5466 hyper
5467 neck
5468 realxe
5469 someine
5470 facial
5471 king_size
547

7148 rrecommend
7149 shed
7150 aintgua
7151 stocked
7152 chatting
7153 output
7154 darkish
7155 friedly
7156 maker
7157 bowel
7158 paned
7159 ammenitie
7160 civilized
7161 beleive
7162 hung
7163 scarily
7164 outrageously
7165 breathless
7166 dissapear
7167 exaggerate
7168 collective
7169 conglomerate
7170 farmer
7171 retaurant
7172 seeing
7173 breathe
7174 mouldy
7175 spiky
7176 friday
7177 prebook
7178 presence
7179 suffocate
7180 perforated
7181 talent
7182 alll
7183 crockery
7184 fascinating
7185 medication
7186 hugely
7187 recouperate
7188 sucluded
7189 accurately
7190 partially
7191 rooter
7192 noisier
7193 advertisement
7194 firewall
7195 prop
7196 rave_reviews
7197 console
7198 duffel
7199 pod
7200 enviroment
7201 lace
7202 scense
7203 maze
7204 wonderment
7205 partys
7206 doozy_koala
7207 barring
7208 evelyan
7209 lab
7210 tanaah
7211 liner
7212 industrial
7213 spoon
7214 furbish
7215 effectively
7216 independence
7217 owl
7218 specie
7219 excercise
7220 swimming
7221 hoover
72

8897 appetizing
8898 shack
8899 abate
8900 screech
8901 tyre
8902 whopping
8903 cereus
8904 cirillo
8905 richardo
8906 archeological
8907 spirituality
8908 comforable
8909 transporation
8910 miniaturize
8911 depression
8912 miserable
8913 saggy
8914 cancellable
8915 compromise
8916 vengeance
8917 brewery
8918 reoutfitte
8919 martini
8920 pup
8921 endure
8922 leafy
8923 ledge
8924 sorely
8925 headboard
8926 upgraded
8927 drapery
8928 refinement
8929 inquiry
8930 courtyarda
8931 sneaker
8932 stupid
8933 instalation
8934 decission
8935 facined
8936 connectivity
8937 anough
8938 hoarder
8939 cab_ride
8940 accumulate
8941 irrelevant
8942 gong
8943 suspiciously
8944 casa_rustica
8945 lame
8946 palicochico
8947 egregiously
8948 legitimate
8949 massively
8950 soulless
8951 prearranged
8952 percent
8953 birds
8954 menagerie
8955 mopping
8956 solvent
8957 perpendicular
8958 metamorphosis
8959 organizer
8960 housemate
8961 kinberly
8962 took
8963 sock
8964 analyze
8965 groundskeeper
8966 freacke


9897 cycling
9898 unavoidable
9899 vacay
9900 scandinavian
9901 barf
9902 guideline
9903 homy
9904 sided
9905 applause
9906 disregard
9907 minimize
9908 raffa
9909 platano
9910 bending
9911 setback
9912 hoise
9913 thw
9914 reccommende
9915 experimentation
9916 trait
9917 anjeline
9918 cork
9919 breakfeast
9920 maket
9921 snow
9922 harvesting
9923 resolution
9924 vigilant
9925 eccentric
9926 macadamia
9927 repellent
9928 valve
9929 ontop
9930 undeniable
9931 yogalesson
9932 slog
9933 childhood
9934 gone
9935 snip
9936 electronics
9937 filch
9938 regenerate
9939 uncomfortably
9940 totilla
9941 harmoniously
9942 icon
9943 memorabilia
9944 oddity
9945 stairs
9946 typewriter
9947 barstaff
9948 marys
9949 booty
9950 aquiped
9951 gutte
9952 cristian
9953 citys
9954 aggravation
9955 alisandra
9956 indulging
9957 janitor
9958 paved
9959 bf
9960 cuteness
9961 greeat
9962 hob
9963 crossing
9964 wondering
9965 tavern
9966 poo
9967 vouch
9968 servoce
9969 eveeything
9970 muffled
9971 tuba
9972 mbit

11397 nightcap
11398 chocolate_factory
11399 mixtrure
11400 swimmer
11401 mutual
11402 ridge
11403 coordinating
11404 innovative
11405 cuality
11406 backgammon
11407 undecided
11408 commenter
11409 fernandoz
11410 celine
11411 pane
11412 aristocrat
11413 qualitatively
11414 blouse
11415 chicos
11416 guaranty
11417 adapter
11418 etca
11419 leyvi
11420 amass
11421 write_review
11422 entryway
11423 obnoxiously
11424 elevation
11425 faciliie
11426 ashen
11427 numbered
11428 deprive
11429 revitalise
11430 curricular
11431 alternately
11432 paie
11433 ammount
11434 possesion
11435 nƒn
11436 nƒnz
11437 greater
11438 pottery
11439 kitschy
11440 iin
11441 charma
11442 coordinator
11443 deciding
11444 esle
11445 chimaltenango
11446 outlying
11447 per_device
11448 trading
11449 uncertainty
11450 thoight
11451 belie
11452 pervade
11453 garb
11454 provision
11455 stint
11456 certanly
11457 avocadoe
11458 unthinkable
11459 tact
11460 diary
11461 whodoesthis
11462 cloudless
11463 brutalist
11464 cont

12960 bannana
12961 curtly
12962 harshly
12963 glossy


In [28]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[1:2])
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1)],
 [(0, 1),
  (1, 1),
  (3, 1),
  (10, 1),
  (12, 1),
  (13, 2),
  (16, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 2),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1)],
 [(1, 1),
  (6, 1),
  (16, 1),
  (50, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1)],
 [(1, 2),
  (6, 1),
  (13, 1),
  (23, 1),
  (38, 1),
  (54, 1),
  (73, 1),
  

In [30]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('amazing', 1),
  ('beautiful', 1),
  ('expectation', 1),
  ('great', 1),
  ('ground', 1),
  ('highly', 1),
  ('hotel', 1),
  ('locate', 1),
  ('night', 1),
  ('perfectly', 1),
  ('private', 1),
  ('recommend', 1),
  ('restaurant', 1),
  ('room', 1),
  ('service', 1),
  ('start', 1),
  ('stay', 1),
  ('suggest', 1),
  ('surpass', 1),
  ('tour', 1),
  ('trip', 1),
  ('visit', 1),
  ('walking', 1),
  ('wonderful', 1)]]

## Latent Dirichlet Allocation

![Latent Dirichlet Allocation](https://miro.medium.com/max/1400/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg)

In [31]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [32]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.361*"feel" + 0.090*"home" + 0.080*"truly" + 0.062*"explore" + '
  '0.055*"stunning" + 0.043*"vacation" + 0.040*"reasonable" + '
  '0.034*"appreciate" + 0.031*"soon" + 0.027*"daily"'),
 (1,
  '0.161*"great" + 0.117*"food" + 0.092*"go" + 0.076*"really" + '
  '0.058*"amazing" + 0.052*"view" + 0.051*"back" + 0.035*"way" + '
  '0.034*"definitely" + 0.030*"always"'),
 (2,
  '0.152*"spa" + 0.084*"special" + 0.069*"romantic" + 0.062*"request" + '
  '0.044*"charm" + 0.037*"luxurious" + 0.033*"rather" + 0.032*"incredibly" + '
  '0.029*"busy" + 0.028*"will"'),
 (3,
  '0.250*"site" + 0.177*"surround" + 0.115*"walking_distance" + '
  '0.071*"customer" + 0.066*"attraction" + 0.061*"management" + '
  '0.044*"delight" + 0.027*"distance" + 0.022*"toilet" + 0.003*"eatery"'),
 (4,
  '0.233*"service" + 0.172*"ground" + 0.068*"spend" + 0.054*"quite" + '
  '0.045*"eat" + 0.045*"keep" + 0.034*"especially" + 0.027*"business" + '
  '0.022*"option" + 0.022*"peaceful"'),
 (5,
  '0.341*"price" + 0.160*"

In [33]:
doc_lda = lda_model[corpus]

In [34]:
#Perplejidad
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Score de coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.143245952220477

Coherence Score:  0.3258889575945736


In [36]:
# Visualizamos los temas
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.329751 -0.131827       1        1  30.344191
11    -0.326302 -0.160514       2        1  23.094133
12    -0.243589  0.170632       3        1   9.352941
10    -0.170717 -0.000636       4        1   6.324319
19     0.043414  0.020790       5        1   5.710669
1     -0.115727 -0.159980       6        1   4.414690
4     -0.131379  0.117948       7        1   3.970877
9     -0.068912  0.037349       8        1   3.209080
7      0.060094 -0.072084       9        1   2.110113
8     -0.001782  0.250970      10        1   2.038592
2     -0.024926  0.224316      11        1   1.941605
16     0.099432 -0.027250      12        1   1.428083
6      0.114616 -0.012908      13        1   1.231923
15     0.124839 -0.027352      14        1   1.064517
5      0.129760 -0.034608      15        1   0.964719
17     0.150819 -0.038396      16        1   0.818114
14     0.153209 -0.016865      17        1   0.742186
3      0.161995 -0.038542      18        1   0.586764
13     0.174362 -0.061456      19        1   0.417525
18     0.200547 -0.039587      20        1   0.234966, topic_info=           Term          Freq         Total Category  logprob  loglift
3700         ax  52208.000000  52208.000000  Default  30.0000  30.0000
200       write  13238.000000  13238.000000  Default  29.0000  29.0000
140      people   7560.000000   7560.000000  Default  28.0000  28.0000
38         year   4725.000000   4725.000000  Default  27.0000  27.0000
17         line  11988.000000  11988.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
4645   somewhat      0.041981      1.364619  Topic20 -11.1121   2.5721
4331    install      0.041981      1.364635  Topic20 -11.1121   2.5720
3202    digital      0.041980      1.364629  Topic20 -11.1121   2.5720
6202  president      0.041980      1.364621  Topic20 -11.1122   2.5720
3175     accord      0.041980      1.364668  Topic20 -11.1122   2.5720

[851 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1212      2  0.998981    accept
583       7  0.996896  activity
40        5  0.989354   adapter
41        1  0.139306       add
41        4  0.389857       add
...     ...       ...       ...
200       1  0.997483     write
200       2  0.002417     write
441      15  0.990523   writing
38        2  0.217334      year
38        4  0.782573      year

[1100 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 12, 13, 11, 20, 2, 5, 10, 8, 9, 3, 17, 7, 16, 6, 18, 15, 4, 14, 19])